# Promi Exercise Sheet 3: Estimation and Evaluation

In [94]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse

# Part 1: Regression
In this exercise, you will work on linear regression with polynomial features where we model the function $f(\bm{x})$ as

$$f(x) = \bm{w}^\intercal \phi(x).$$

The true model is a polynomial of degree 3

$$f(x) = 0.5 + (2x - 0.5)^2 + x^3$$

We further introduce noise into the system by adding a noise term $\varepsilon_i$ which is sampled from a Gaussian distribution

$$y = f(x) + \varepsilon_i, \quad\varepsilon_i\sim \mathcal{N}(0, \sigma^2).$$

In [95]:
def f(x):
  """The true polynomial that generated the data D
  Args:
    x: Input data
  Returns:
    Polynomial evaluated at x
  """
  return x ** 3 + (2 * x - .5) ** 2 + .5

def generate_data(n, minval, maxval, variance=1., train=False, seed=0):
  """Generate the datasets. Note that we don't want to extrapolate,
  and such, the eval data should always lie inside of the train data.
  Args:
    n: Number of datapoints to sample. n has to be atleast 2.
    minval: Lower boundary for samples x
    maxval: Upper boundary for samples x
    variance: Variance or squared standard deviation of the model
    train: Flag deciding whether we sample training or evaluation data
    seed: Random seed
  Returns:
    Tuple of randomly generated data x and the according y
  """
  # Set numpy random number generator
  rng = np.random.default_rng(seed)

  # Sample data along the x-axis
  if train:
    # We first sample uniformly on the x-Axis
    x = rng.uniform(minval, maxval, size=(n - 2,))
    # We will sample on datapoint beyond the min and max values to
    # guarantee that we do not extrapolate during the evaluation
    margin = (maxval - minval) / 100
    min_x = rng.uniform(minval - margin, minval, (1,))
    max_x = rng.uniform(maxval, maxval + margin, (1,))
    x = np.concatenate((x, min_x, max_x))
  else:
    x = rng.uniform(minval, maxval, size=(n,))
  eps = rng.standard_normal(n)

  #
  return x, f(x) + variance * eps

## Question 1.1: Least Squares Linear Regression
To carry out regression, we first need to define the basis functions $\phi(\mathbf{x})$. In this task we would like to use polynomial features of degree $k$. Furthermore, we implement ridge regression as we want to counteract overfitting.


In [96]:
def polynomial_features(x, degree):
  """
  A polynomial feature function of degree n.
  :param x: input of size (N,)
  :param degree: polynomial degree
  :return: Polynomial features evaluated at x of dim (degree, N)
  """
  ### BEGIN SOLUTION
  features = np.zeros((degree + 1, x.shape[0]))
  for i in range(degree + 1):
    features[i] = x ** i
  return features
  ### END SOLUTION

def fit_w(x, y, lam, degree):
  """
  Fit the weights with the closed-form solution of ridge regression.
  Args:
    x: Input of size (N,)
    y: Output of size (N,)
    lam: Regularization parameter lambda
    degree: Polynomial degree
  Returns:
    Optimal weights
  """
  ### BEGIN SOLUTION
  features = polynomial_features(x, degree)
  M = features.shape[0]
  kernel = features @ features.T # np.einsum("ij,kj->ik", features, features)
  inverse = np.linalg.inv(kernel + lam * np.eye(M))
  return inverse @ features @ y # np.einsum("ij,jk,k->i", inverse, features, y)
  ### END SOLUTION

def predict(x, w, degree):
  """
  Calculate the generalized linear regression estimate w^T phi(x) given x,
  the feature function, and weights w.
  Args:
    x: input of size (N,)
    w: Weights of size (M)
    degree: Polynomial degree
  Returns:
    Generalized linear regression estimate
  """
  ### BEGIN SOLUTION
  features = polynomial_features(x, degree)
  return w.T @ features #np.einsum("i,ij->j", w, features)
  ### END SOLUTION

def calc_mse(x, y):
  """
  Calculates the mean squared error (MSE) between x and y
  Args:
    x: Data x of size (N,)
    y: Data y of size (N,)
  Returns:
    MSE
  """
  ### BEGIN SOLUTION
  return np.mean((x-y) ** 2)
  ### END SOLUTION

Here you can try out your code by simply running the following cell. This cell will carry out your ridge regression implementation from above for $\lambda=0$ in which case we are provided with the linear least squares solution.

We evaluate the regression task on six different polynomial sizes $k = \{0,1,3,5,7,9\}$ based on your implementation of the MSE.

In [ ]:
%matplotlib inline

# Settings
n_train = 15
n_test = 100
minval = -2.
maxval = 2


train_data = generate_data(n_train, minval, maxval, train=True, seed=1001)
test_data = generate_data(n_test, minval, maxval, train=False, seed=1002)

def plot_linear_regression(x, y, labels, eval_quantity):
  """Plotting functionality for the prediction of linear regression
  for K different polynomial degrees.
  Args:
    x: Data of size (K, N). The first dimension denotes the different
      polynomial degrees that has been experimented with
    y: Data of size (K, N)
  """
  K = x.shape[0]
  colors = matplotlib.colormaps['Reds'].resampled(K+1)(range(1, K+1))
  fig = plt.figure()
  plt.scatter(test_data[0], test_data[1], color="tab:orange", linewidths=0.5, label="Test", alpha=0.3)
  plt.scatter(train_data[0], train_data[1], color="tab:blue", linewidths=0.5, label="Train", alpha=0.3)
  for i in range(K):
    plt.plot(x[i], y[i], label=f"{eval_quantity}={labels[i]}", color=colors[i], lw=2.5)
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title('Data Points and Regression Curves')
  plt.legend()

def plot_mse(mse, labels):
  """Plotting functionality of the MSE for K different polynomial degrees."""
  fig = plt.figure()
  plt.plot(labels, mse)
  plt.scatter(labels, mse)
  plt.xticks(labels)
  plt.ylabel("MSE")
  plt.xlabel("Polynomial Degree")
  plt.title("Mean Squared Error for Different Polynomial Degrees")

# Evaluate regression for different polynomial degrees
degrees = [0, 1, 3, 5, 7, 9]
xs, ys, mse = [], [], []
for degree in degrees:
  w = fit_w(
      train_data[0], train_data[1],
      lam=0., # Edge case resulting in linear least squares regression
      degree=degree
  )
  # Predict the test data
  y_test = predict(test_data[0], w, degree)
  mse.append(calc_mse(y_test, test_data[1]))

  # Run regression over the whole interval
  xs.append(np.linspace(minval, maxval, 100))
  ys.append(predict(xs[-1], w, degree))
xs = np.stack(xs)
ys = np.stack(ys)

plot_linear_regression(xs, ys, labels=degrees, eval_quantity="k")
plot_mse(mse, degrees)

## Question 1.2: Interpretation of the results
Please describe your results below in a few lines thereby answering which model you would choose and which phenomenon we see for small and large polynomial degrees.

### BEGIN SOLUTION
The mean squared error is minimized for the polynomial degrees of 3, 5, and 7. The observation fits well with the definition of the true function being of polynomial degree 3. For smaller polynomial degrees, the features are not expressive enough to predict the non-linearity of the data - underfitting. For larger polynomial degrees we can see overfitting.
### END SOLUTION

## Question 1.3: Bias Variance Tradeoff in Linear Regression

The MSE can be used to quantify the performance of our estimator, $\hat{f}$. The MSE can be decomposed into a form involving the bias and variance, giving rise to the bias-variance tradeoff. Suppose we would like to fit a model to regress on a query datapoint ($y(\bm{x}_q), \bm{x}_q$):

$$
y(\bm{x}_q) = f(\bm{x}_q) + \epsilon
$$

where $\mathbb{E}[\epsilon] = 0$ and $Var[\epsilon] = \sigma_\epsilon^2$. The loss is quantified as 

$$
\begin{align*}
\mathcal{L}_{\hat{f}}(\bm{x}_q) &=  \mathbb{E}_{\mathcal{D}, \epsilon} \left[ (y(\bm{x}_q) - \hat{f}_{\mathcal{D}}(\bm{x}_q))^2 \right] \\
\end{align*}
$$

Show that this can be reduced to sum of the bias and variance, i.e.:

$$
\mathcal{L}_{\hat{f}}(\bm{x}_q) = \sigma^2_{\epsilon} + \text{bias}^2 + \text{var}
$$

### BEGIN SOLUTION

We begin by adding and subtracting the expected estimator $f = \mathbb{E}[y(\bm{x}_q)]$:

$$
\mathcal{L}_{\hat{f}}(\bm{x}_q) = \mathbb{E}_{\mathcal{D}, \epsilon} \left[ \left( y(\bm{x}_q) - f(\bm{x}_q) + f(\bm{x}_q) - \hat{f}_{\mathcal{D}}(\bm{x}_q) \right)^2 \right]
$$

Next, we expand using the property $(a + b)^2 = a^2 + b^2 + 2ab$,  with
- $a= y(\bm{x}_q) - f(\bm{x}_q)$
- $b= f(\bm{x}_q) - \hat{f}_{\mathcal{D}}(\bm{x}_q)$

$$
\mathcal{L}_{\hat{f}}(\bm{x}_q) = \mathbb{E}_{\mathcal{D}, \epsilon} \left[ \Big( y(\bm{x}_q) - f(\bm{x}_q) \Big)^2 + \Big(f(\bm{x}_q) - \hat{f}_{\mathcal{D}}(\bm{x}_q) \Big)^2 + 2\Big( \Big(y(\bm{x}_q) - f(\bm{x}_q)\Big) \Big(f(\bm{x}_q) -  \hat{f}_{\mathcal{D}}(\bm{x}_q) \Big) \Big) \right]
$$

Because $\mathbb{E}[\epsilon] = 0$ and $\epsilon = y(\bm{x}_q) - f(\bm{x}_q)$, the last term:

$$
2 \epsilon \Big(f(\bm{x}_q) -  \hat{f}_{\mathcal{D}}(\bm{x}_q) \Big)
$$

vanishes, and the first term is the variance of $\epsilon$, that is: $\sigma_\epsilon^2$. This leaves us with:

$$
\mathcal{L}_{\hat{f}}(\bm{x}_q) = \sigma_\epsilon^2 + \mathbb{E}_{\mathcal{D}} \left[\Big(f(\bm{x}_q) -  \hat{f}_{\mathcal{D}}(\bm{x}_q) \Big)^2 \right]
$$

Using $\bar{\hat{f}}(\bm{x}_q) = \mathbb{E}_\mathcal{D} \left[ \hat{f}_\mathcal{D}(\bm{x}_q) \right]$, we obtain:

$$
\begin{align*}
\mathbb{E}_{\mathcal{D}} \left[\left(f(\bm{x}_q) -  \hat{f}_{\mathcal{D}}(\bm{x}_q)\right)^2 \right] &= \mathbb{E}_{\mathcal{D}} \left[ \left( f(\bm{x}_q) - \bar{\hat{f}}(\bm{x}_q) + \bar{\hat{f}}(\bm{x}_q) - \hat{f}_{\mathcal{D}}(\bm{x}_q) \right)^2 \right] \\
&= \mathbb{E}_{\mathcal{D}}\left[f(\bm{x}_q) -  \bar{\hat{f}}(\bm{x}_q)\right]^2 +  \mathbb{E}_{\mathcal{D}} \left[\left(\bar{\hat{f}}(\bm{x}_q) -  \hat{f}_{\mathcal{D}}(\bm{x}_q)\right)^2 \right] \\
&= \text{bias}^2 + \text{var}
\end{align*}
$$

by using the same argument as done above.

### END SOLUTION

## Question 1.4: Empirical evaluation of the bias and the variance
The bias-variance tradeoff is typically a purely theoretical concept as it requires the evaluation of $f(x)$ which is commonly not available. In this task we assume that $f(x)$ is known and thus, an approximation of the bias and variance is possible. Instead of the pointwise bias and variance based on a query $\bm{x}_q$, we introduce the lecture bias and variance as an expected value over an evaluation dataset $\mathcal{D}^\prime$.
$$
\begin{aligned}
    \text{Bias}(\hat{f}) &= \mathbb{E}_{\bm{x}_q \sim \mathcal{D^\prime}}[\text{bias}^2(\hat{f}(\bm{x}_q))] &&= \mathbb{E}_{\bm{x}_q \sim \mathcal{D^\prime}}[f(\bm{x}_q) - \bar{\hat{f}}(\bm{x}_q)],\\
    \text{Var}(\hat{f}) &= \mathbb{E}_{\bm{x}_q \sim \mathcal{D^\prime}}[\text{var}(\hat{f}(\bm{x}_q))] &&= \mathbb{E}_{\bm{x}_q \sim \mathcal{D^\prime}}[\mathbb{E}_{\tilde{\mathcal{D}} \subset\mathcal{D}}[(\bar{\hat{f}}(\bm{x}_q) - \hat{f}_{\tilde{\mathcal{D}}}(\bm{x}_q))^2]].
\end{aligned}
$$
Here, $\bar{\hat{f}}$ is the average prediction of the maximum likelihood estimator over various datasets sampled from the ground truth distribution
$$\bar{\hat{f}}(\bm{x}_q) = \mathbb{E}_{\tilde{\mathcal{D}} \subset\mathcal{D}}[f_{\tilde{\mathcal{D}}}(\bm{x}_q)].$$

To empirically study the effect of the Bias and the Var, we approximate the expectations with Monte Carlo samples as follows
$$\bar{\hat{f}}(x) \approx \frac{1}{M}\sum_{j=1}^{M}\left(\hat{f}_{\mathcal{D_j}}(x)\right).$$

$$\text{Bias}[\hat{f}_\mathcal{D}] \approx \frac{1}{N}\sum_{i=1}^{N}\left(f(x_i) - \bar{\hat{f}}(x_i)\right)^2,$$

$$\text{Var}\left[\hat{f}_\mathcal{D}\right] \approx \frac{1}{NM}\sum_{i=1}^{N}\sum_{j=1}^{M}\left(\hat{f}_{\mathcal{D}_j}(x_i) - \bar{\hat{f}}(x_i)\right)^2$$

Note that the indice $j=1,\dots,M$ refer to subsets $\tilde{\mathcal{D}}_i$ from the dataset $\mathcal{D}$, whereas the samples $x_i$ from the outer expectation relate to the indices denoted by $i=1,\dots,N$ from an evaluation dataset $\mathcal{D}^\prime$.

In the following exercise, please implement the Monte Carlo approximations for the average prediction, the Bias, and the Variance as shown above.

Hint: You can and absolutely should use the `predict` function from the regression example as well as the function of the true model `f` from above.

In [98]:
def avg_prediction(x, ws, degree):
  """
  Approximation of the average prediction using the M function estimations
  Args:
    x: Input data of size (N,). N denotes the number of samples in the evaluation dataset.
    ws: The weights obtained from ridge regression of size (M, degree). 
      The first dimension denotes the M different subset datasets on which the estimator is fitted.
    degree: The polynomial degree
  Returns:
    The average prediction as a scalar
  """
  ### BEGIN SOLUTION
  predictions = []
  for w in ws:
    pred = predict(x, w, degree)
    predictions.append(pred)
  return np.stack(predictions).mean(axis=0)
  ### END SOLUTION


def calc_bias(x_q, ws, degree):
  """Estimate the bias.
  Args:
    x_q: Input data of size (N,). N denotes the number of samples in the evaluation dataset.
    ws: The weights obtained from ridge regression of size (M, degree)
      The first dimension denotes the M different subset datasets on which the estimator is fitted.
    degree: The polynomial degree
  Returns:
    Model Bias as a scalar
  """
  ### BEGIN SOLUTION
  function_eval = f(x_q)
  avg_pred = avg_prediction(x_q, ws, degree)
  return np.mean((function_eval - avg_pred) ** 2)
  ### END SOLUTION

def calc_variance(x_q, ws, degree):
  """Estimate the model variance
  Args:
    x_q: Input data of size (N,). N denotes the number of samples in the evaluation dataset.
    ws: The weights obtained from ridge regression of size (M, degree)
      The first dimension denotes the M different subset datasets on which the estimator is fitted.
    degree: The polynomial degree
  Returns:
    Model variance as a scalar
  """
  ### BEGIN SOLUTION
  avg_pred = avg_prediction(x_q, ws, degree)
  mean = []
  for w in ws:
    pred = predict(x_q, w, degree)
    mean.append(((pred - avg_pred) ** 2))
  return np.mean(np.stack(mean))
  ### END SOLUTION

You can test your implementation by running the below coding snippet. It estimate the bias and variance for $M = 25$ datasets with each dataset containing $N=20$ datapoints.

In [ ]:
%matplotlib inline

# Settings
n = 20
m = 25
minval = -2.
maxval = 2.
degree = 9
train_datasets = []
seed = 3001
for i in range(m):
  train_datasets.append(generate_data(n_train, minval, maxval, train=True, seed=seed))
  seed += 1
eval_points = np.linspace(minval, maxval, n)

# Estimate the bias and variance
biases = []
vars = []
xs, ys = [], []
lambdas = [1e-6, 1e-4, 1e-2, 1e0, 1e2, 1e4, 1e6]
# Evaluate the different models, characterized by the regularization parameter lambda
for l in lambdas:
  w_maps = []
  for data in train_datasets:
    # Fit the estimator on each individual dataset
    w = fit_w(x=data[0], y=data[1], lam=l, degree=degree)
    w_maps.append(w)
  # Estimate the bias of the model
  bias = calc_bias(eval_points, w_maps, degree)
  biases.append(bias)
  # Estimate the variance of the model
  var = calc_variance(eval_points, w_maps, degree)
  vars.append(var)
  xs.append(np.linspace(minval, maxval, 100))
  ys.append(predict(xs[-1], w_maps[0], degree))

biases = np.array(biases)
vars = np.array(vars)
xs = np.stack(xs)
ys = np.stack(ys)

# Plot the bias and variance for different lambas
plt.figure()
plt.plot(lambdas, biases, label="Bias")
plt.plot(lambdas, vars, label="Variance")
plt.plot(lambdas, biases + vars, label="Total Err.")
plt.xscale("log")
plt.xlabel(r"$\lambda$")
plt.ylabel("Error")
plt.legend()
plt.title("Total Error for Different Regularization Values")

# Calculate predictions
plot_linear_regression(xs, ys, labels=lambdas, eval_quantity=r"$\lambda$")

## Question 1.5: Interpretation of the bias and variance trade-off
Please explain the results in a few sentences. In particular, provide an explanation if the bias and variance behave as expected. For which regularization parameter $\lambda$ would you decide?

### BEGIN SOLUTION
If we choose very small regularization parameters $\lambda$, the model tends to overfit to the data, whereas for large $\lambda$, the model tends to underfit the data. As the polynomial degree is higher than the true function, the model is prone to overfit which can be seen when we do not regularize the model (small $\lambda$). The overfitting correlates with a high variance as the optimization can get stuck in multiple minima. On the contrary, high regularization restricts the expressive power of the model, and thus yields a high bias.
### END SOLUTION

# Part 2: Clustering using Parametric Density Estimation

## Question 2.1: Visualization of the Dataset
Before being able to cluster the dataset, you need to obtain an initial impression of the data to cluster. Please generate a scatter plot of the data. What is the optimal number of clusters for the dataset?

In [ ]:
### BEGIN SOLUTION

# Load dataset from .npy file
data = np.load('./clustering_dataset.npy')

# Plot the dataset
plt.scatter(data[:, 0], data[:, 1])
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Dataset Visualization')
plt.show()

### END SOLUTION

Please set the optimal number of clusters based on your empirical observation of the dataset.

In [101]:
OPTIMAL_NUM_CLUSTERS = (
    ### BEGIN SOLUTION
    3
    ### END SOLUTION
)

## Question 2.2: Implementation of the EM Algorithm
In the lecture about parametric density estimation, you got introduced to the EM Algorithm to be able to fit a Gaussian Mixture Model (GMM) over your dataset. Please implement the EM algorithm from scratch by filling in the following functions.

In [102]:
def e_step(data, mixture_weights, means, covariances):
    """
    :param data: all data points
    :param mixture_weights: mixture weights of Gaussian Mixture Model
    :param means: means of Gaussian Mixture Model
    :param covariances: covariances of Gaussian Mixture Model

    :return: responsibilities of shape (num_data_samples, num_clusters)
    """
    ### BEGIN SOLUTION
    num_data_samples, num_dimensions = data.shape
    num_clusters = len(mixture_weights)

    # Initialize the responsibility matrix
    responsibilities = np.zeros((num_data_samples, num_clusters))

    for n in range(num_data_samples):
        for k in range(num_clusters):
            covariance = covariances[:, :, k]
            covariance_det = np.linalg.det(covariance)
            diff = data[n] - means[k]
            coefficient = 1. / float(((2 * np.pi) ** (float(num_dimensions) / 2)) * np.sqrt(covariance_det))
            exponent = np.exp(-0.5 * np.dot(diff.T, np.linalg.lstsq(covariance, diff, rcond=-1)[0]))

            responsibilities[n, k] = mixture_weights[k] * coefficient * exponent
        responsibilities[n] /= responsibilities[n].sum()

    return responsibilities

    ### END SOLUTION

In [103]:
def m_step(data, responsibilities):
    """
    :param data: all data points
    :param responsibilities: responsibilities from e_step

    :return: updated mixture_weights, updated means, updated covariances
    """
    ### BEGIN SOLUTION
    num_data_samples, num_dimensions = data.shape
    num_clusters = responsibilities.shape[1]

    # Create matrices
    new_covariances = np.zeros((num_dimensions, num_dimensions, num_clusters))

    N_k = responsibilities.sum(axis=0)
    new_mixture_weights = N_k / num_data_samples

    new_means = np.divide(np.dot(responsibilities.T, data), N_k[:, np.newaxis])

    for k in range(num_clusters):
        aux_covariance = np.zeros((num_dimensions, num_dimensions))
        for n in range(num_data_samples):
            diff = data[n] - new_means[k]
            aux_covariance = aux_covariance + responsibilities[n, k] * np.outer(diff, diff.T)
        new_covariances[:, :, k] = aux_covariance / N_k[k]

    return new_mixture_weights, new_means, new_covariances
    ### END SOLUTION

In [104]:
def log_likelihood(data, mixture_weights, means, covariances):
    """
    :param data: all data points
    :param mixture_weights: mixture weights of Gaussian Mixture Model
    :param means: means of Gaussian Mixture Model
    :param covariances: covariances of Gaussian Mixture Model

    :return: calculated log likelihood
    """

    ### BEGIN SOLUTION
    num_data_samples, num_dimensions = data.shape
    num_clusters = len(mixture_weights)

    log_likelihood = 0.0

    for n in range(num_data_samples):
        likelihood = 0.0
        for k in range(num_clusters):
            covariance = covariances[:, :, k]
            covariance_det = np.linalg.det(covariance)
            diff = data[n] - means[k]
            coefficient = 1. / float(((2 * np.pi) ** (float(num_dimensions) / 2)) * np.sqrt(covariance_det))
            exponent = np.exp(-0.5 * np.dot(diff.T, np.linalg.lstsq(covariance, diff, rcond=-1)[0]))
            likelihood += mixture_weights[k] * coefficient * exponent

        log_likelihood += np.log(likelihood)

    return log_likelihood

    ### END SOLUTION

Now, implement the expectation_maximization function which leverages the previously implemented functions. Given initial values of the mixture weights, means and covariances, you should apply the E- and M-Step until the log-likelihood has converged, i.e. the absolute difference in subsequent values is below a certain threshold.

In [105]:
def expectation_maximization(data, init_mixture_weights, init_means, init_covariances, threshold):
    """
    :param data: all data points
    :param init_mixture_weights: initial mixture weights of Gaussian Mixture Model
    :param init_means: initial means of Gaussian Mixture Model
    :param init_covariances: initial covariances of Gaussian Mixture Model
    :param threshold: threshold for checking convergence

    :return: final mixture_weights, final means, final covariances, final responsibilities, final log_likelihood, num_iterations needed till convergence
    """
    ### BEGIN SOLUTION
    mixture_weights = init_mixture_weights
    means = init_means
    covariances = init_covariances

    # Evaluate initial value of log likelihood
    previous_log_likelihood = log_likelihood(data, mixture_weights, means, covariances)

    # Iteration counter
    iteration = 0

    while True:
        # E-step: Compute responsibilities
        responsibilities = e_step(data, mixture_weights, means, covariances)

        # M-step: Update parameters
        mixture_weights, means, covariances = m_step(data, responsibilities)

        # Compute log likelihood
        current_log_likelihood = log_likelihood(data, mixture_weights, means, covariances)

        log_likelihood_diff = np.abs(current_log_likelihood - previous_log_likelihood)
        print("Training Info: Iteration {} Log Likelihood difference  {}".format(iteration, log_likelihood_diff))

        if log_likelihood_diff < threshold:
            break

        previous_log_likelihood = current_log_likelihood
        iteration += 1

    return mixture_weights, means, covariances, responsibilities, current_log_likelihood, iteration

    ### END SOLUTION

## Question 2.3: Clustering the Dataset and Visualizing the Results


In [106]:
def plot_results(data, responsibilities, means, covariances):
    """
    :param data: all data points
    :param responsibilities: final responsibilities
    :param means: final means
    :param covariances: final covariances

    :return: None
    """

    num_components = len(means)
    fig, ax = plt.subplots()

    # label the data points w.r.t to the final responsibilities
    labels = np.argmax(responsibilities, axis=1)
    ax.scatter(data[:, 0], data[:, 1], c=labels)

    # plot the resulting ellipsoids of mixture of Gaussians
    for k in range(num_components):
        eigenvalues, eigenvectors = np.linalg.eig(covariances[:,:,k])

        # Sort eigenvalues and eigenvectors in descending order
        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[sorted_indices]
        eigenvectors = eigenvectors[:, sorted_indices]

        # Calculate the angle of rotation
        angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))

        # Create an ellipse based on the eigenvalues and eigenvectors
        ellipse = Ellipse(means[k], 2 * np.sqrt(eigenvalues[0]), 2 * np.sqrt(eigenvalues[1]),
                          angle=angle, fill=False, edgecolor='r')

        # Plot the ellipse
        ax.add_patch(ellipse)

    # Add labels and title
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Clustering Result')
    plt.grid(True)
    plt.show()

Please implement the main function which should load the dataset, execute the EM algorithm and plot the results using the previously implemented functions. Note, the EM algorithm expects initial values for the mixture weights, means and covariances of the Gaussian Mixture Model. Please initialize the mixture weights uniformly over the number of mixture components and the initial covariances with a unit scale. You can experiment around with the initial means as you like.

In [ ]:
DATASET_PATH = "./clustering_dataset.npy"

# Hyperparameters
NUM_MIXTURE_COMPONENTS = OPTIMAL_NUM_CLUSTERS             # as previously defined by yourself
CONVERGENCE_THRESHOLD = 1e-6


def main():
    """
    :return: None
    """
    ### BEGIN SOLUTION

    data = np.load(DATASET_PATH)
    num_training_samples, num_dimensions = data.shape

    init_mixture_weights = np.ones(NUM_MIXTURE_COMPONENTS) / NUM_MIXTURE_COMPONENTS
    init_means = np.array([[2.5, 0], [6.0, 5.0], [7.5, -7.5]])
    init_covariances = np.stack([np.eye(num_dimensions)] * NUM_MIXTURE_COMPONENTS, axis=-1)

    mixture_weights, means, covariances, responsibilities, final_log_likelihood, iteration = expectation_maximization(data, init_mixture_weights, init_means, init_covariances, CONVERGENCE_THRESHOLD)

    print("\n Final weights:", mixture_weights)
    print("\n Final cluster means:", means)
    print("\n Final cluster covariance 1:", covariances[:, :, 0])
    print("\n Final cluster covariance 2:", covariances[:, :, 1])
    print("\n Final cluster covariance 3:", covariances[:, :, 2])
    print("\n Final log likelihood:", final_log_likelihood)
    print("\n EM converged after {} iterations".format(iteration))

    plot_results(data, responsibilities, means, covariances)

    ### END SOLUTION

main()

## Question 2.4: Report Results
Please report the following quantitative information regarding your final solution:
- Final Mixture Weights
- Initial Means
- Final Means
- Final Covariances
- Final Log Likelihood
- Number of Iterations until Convergence

### BEGIN SOLUTION
See above
### END SOLUTION

# Part 3: Biased estimators
For a set of i.i.d. observations $X = \{x_1, x_2, \dots, x_n\}$ from a normal distribution $\mathcal{N}(\mu, \sigma^2)$ with mean $\mu$ and variance $\sigma^2$, we know that the maximum likelihood estimate of the variance is
$$
\hat{\sigma}^2 = \frac{1}{N} \sum_{i=1}^{N} (x_i - \overline{x})^2,
$$
where $\overline{x} = \frac{1}{N} \sum_{i=1}^{N} x_i$ is the sample mean.

## Question 3.1: Bias of the sample variance
Please follow the steps below to show that $\hat{\sigma}^2$ is a biased estimate of the variance. 

1. Derive the expected value of $\hat{\sigma}^2$, i.e., $\mathbb{E}_{X}[\hat{\sigma}^2]$.
2. Show that $\mathbb{E}_{X}[\hat{\sigma}^2] \neq \sigma^2$, indicating that the sample variance tends to underestimate the population variance.
3. Modify the estimator to obtain an unbiased estimate of the population variance.

### BEGIN SOLUTION

1. We are tasked with finding the expected value of the sample variance estimator $\hat{\sigma}^2$, which is given by:

$$
\hat{\sigma}^2 = \frac{1}{N} \sum_{i=1}^{N} (x_i - \overline{x})^2
$$

We can expand this expression as follows:

$$
\hat{\sigma}^2 = \frac{1}{N} \mathbb{E} \left[ \sum_{i=1}^{N} \left( x_i^2 - 2 x_i \overline{x} + \overline{x}^2 \right) \right]
$$

Now, separate the sums:

$$
\hat{\sigma}^2 = \frac{1}{N} \mathbb{E} \left[ \sum_{i=1}^{N} x_i^2 - 2 \sum_{i=1}^{N} x_i \overline{x} + N \overline{x}^2 \right]
$$

We can simplify this further by noting the following identities:

- $\sum_{i=1}^{N} x_i = N \overline{x}$ (since $\overline{x}$ is the sample mean),
- $\sum_{i=1}^{N} \overline{x}^2 = N \overline{x}^2$ (since $\overline{x}$ is constant).

Substitute these into the expression:

$$
\hat{\sigma}^2 = \frac{1}{N} \mathbb{E} \left[ \sum_{i=1}^{N} x_i^2 - 2N \overline{x}^2 + N \overline{x}^2 \right]
$$

Simplifying further:

$$
\hat{\sigma}^2 = \frac{1}{N} \mathbb{E} \left[ \sum_{i=1}^{N} x_i^2 - N \overline{x}^2 \right]
$$

This simplifies to:

$$
\hat{\sigma}^2 = \frac{1}{N} \mathbb{E} \left[ \sum_{i=1}^{N} x_i^2 \right] - \mathbb{E} \left[ \overline{x}^2 \right]
$$

We know that:

- $\mathbb{E}[x_i^2] = \sigma_x^2 + \mu^2$ for each $x_i$, where $\mu$ is the population mean.
- $\mathbb{E}[\overline{x}] = \mu$.

Therefore, the first term becomes:

$$
\frac{1}{N} \mathbb{E} \left[ \sum_{i=1}^{N} x_i^2 \right] = \frac{1}{N} \sum_{i=1}^{N} \left( \sigma_x^2 + \mu^2 \right) = \sigma_x^2 + \mu^2
$$

For the second term, we calculate $\mathbb{E}[\overline{x}^2]$:

$$
\mathbb{E}[\overline{x}^2] = \text{Var}[\overline{x}] + \mathbb{E}[\overline{x}]^2
$$

Since $\overline{x} = \frac{1}{N} \sum_{i=1}^{N} x_i$, the variance of $\overline{x}$ is:

$$
\text{Var}[\overline{x}] = \frac{1}{N^2} \text{Var} \left[ \sum_{i=1}^{N} x_i \right] = \frac{1}{N} \sigma_x^2
$$

Thus:

$$
\mathbb{E}[\overline{x}^2] = \frac{\sigma_x^2}{N} + \mu^2
$$

Substituting these results back, we get:

$$
\hat{\sigma}^2 = \sigma_x^2 + \mu^2 - \left( \frac{\sigma_x^2}{N} + \mu^2 \right)
$$

Simplifying:

$$
\hat{\sigma}^2 = \sigma_x^2 \left( 1 - \frac{1}{N} \right)
$$

Thus, the expected value of the sample variance is:

$$
\mathbb{E}[\hat{\sigma}^2] = \frac{N-1}{N} \sigma_x^2
$$

2. This shows that $\mathbb{E}[\hat{\sigma}^2]$ is slightly smaller than $\sigma_x^2$, meaning the sample variance tends to underestimate the true population variance.

3. To correct for this bias, we modify the estimator to:

$$
\hat{\sigma}^2_{\text{unbiased}} = \frac{1}{N-1} \sum_{i=1}^{N} (x_i - \overline{x})^2
$$

This modification ensures that the expectation of the sample variance is exactly equal to the population variance, $\sigma_x^2$.

### END SOLUTION

# Part 4: Classification

The lecture introduces logistic regression as a solution to a binary classification problem in which the class posterior is modeled by a linear discriminant model and a sigmoid activation function $p(C_1 | \bm{x}) = \sigma(\bm{w}^\intercal \bm{x} + b)$. Here $w$ denotes the weight of the linear function and $b$ the shift in y-direction. Instead of following classical maximum likelihood ideas to estimate the parameters $w$ and $b$, we want to implement a very simple, yet powerful, binary classification algorithm - The perceptron algorithm. Instead of the sigmoid function, we instead use the sign function
$$
\mathrm{sign}(\bm{x}) =\begin{cases}
    1, \quad \mathrm{if}~\bm{x}>0, \\
    0, \quad \mathrm{if}~\bm{x}=0, \\
    -1, \quad \mathrm{if}~\bm{x}<0.
\end{cases}
$$
to directly estimate the class using the discriminant function $y(x) = \mathrm{sign}(\bm{w}^\intercal \bm{x} + b)$.

To introduce the problem, we First generate some synthetic data, where each datapoint belongs to on of two classes (-1 or 1). We further highlight the optimal discriminant function in green.

In [108]:
def plot_classification(data, w, b, classes):
    """Helper function for plotting the data and decision boundary"""

    x_min, x_max = min(data[:, 0]), max(data[:, 0])
    y_min, y_max = min(data[:, 1]), max(data[:, 1])

    x = np.linspace(x_min, x_max)
    if w[1] == 0:
        plt.plot(x, np.ones_like(x) * -b, color="green", label="decision boundary")
    else:
        plt.plot(x, (-b -w[0] * x) / w[1], color="green", label="decision boundary")
    plt.scatter(data[classes, 0], data[classes, 1], color="blue", label="Class 1")
    plt.scatter(data[~classes, 0], data[~classes, 1], color="red", label="Class 2")
    plt.xlim((x_min, x_max))
    plt.ylim((y_min, y_max))
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
data = (2 * np.random.random(size=(50, 2)) - 1) * 10
w_theoric = np.array([1, 2])
b_theoric = 2

classes = (data @ w_theoric.T + b_theoric) > 0
labels = 2 * classes.astype(int) - 1

plot_classification(data, w_theoric, b_theoric, classes)

## Question 4.1: The perceptron algorithm

The perceptron algorithm is a simple algorithm to estimate the weights $w$ and $b$ for the perceptron discriminant function $y(\bm{x}) = \mathrm{sign}(\bm{w}^\intercal \bm{x} + b)$.

Above we plot the data as well as the ground truth linear decision boundary which was used to generate the data.Naturally the question arises "If we don't know the ground truth paramethers, how can we find them?"
For this, you should implement the perceptron algorithm within the next function.

### Perceptron algorithm
- Inititalize the weight $w$ and bias $b$
- For all pairs of data points $(\bm{x}_i, y_i)$, repeat until convergence
    - If $x_i$ is correctly classified, i.e., $y(\bm{x}_i) = y_i$, do nothing,
    - else if $y_i = 1$ update the parameters with $\bm{w} \leftarrow \bm{w} + \gamma \bm{x}_i, b \leftarrow b + 1$,
    - else if $y_i = -1$ update the parameters with $\bm{w} \leftarrow \bm{w} - \gamma \bm{x}_i, b \leftarrow b - 1$

Here, $\gamma$ ia a learning rate which balances how much the weight is moved towards the sample $x_i$.

In [ ]:
def perceptron(data, labels, iterations=100, learning_rate=0.1):
    
    w = np.zeros(2)
    b = 0

    print('Initial')
    plot_classification(data, w, b, classes)

    for i in range(iterations):
        
        ### BEGIN SOLUTION
        random_idx = np.random.randint(0, len(data))
        x, y = data[random_idx], labels[random_idx]
        pred = x @ w + b
        
        if y == 1 and pred <= 0:
            w += learning_rate * x
            b += 1
        elif y == -1 and pred >= 0:
            w -= learning_rate * x
            b -= 1

        ### END SOLUTION

    print('Final solution')
    plot_classification(data, w, b, classes)
    return w, b

w,b = perceptron(data, labels, 10000)

### Question 4.2: Intuition on the update steps
Explain intuitively how the update steps in the perceptron algorithm are similar to gradient descent, and why they help the model improve its classification decision over time. 

### BEGIN SOLUTION
The updates rotate and shift the hyperplane spaned by $\bm{w}$ and $b$ in the case of mis-classification. 

The updates can be thought of as the gradients, $\nabla_{\bm{w}}\mathcal{L}(\bm{w}, b)$ and $\nabla_{b}\mathcal{L}(\bm{w}, b)$, of the classification loss
$$\mathcal{L}(\bm{w}, b) = \max(0, -y_i (\bm{w}^\intercal \bm{x}_i + b)).$$
Thus, the perceptron algorithm can be seen as a first order gradient descent optimization method.
### END SOLUTION